# Time to Trial End
Determine the value of hyperparameter `cnfg.TIME_TO_TRIAL_END_THRESHOLD`, which is used to decide if a fixation / visit is a _LWS_ instance or not.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import config as cnfg

# pio.renderers.default = "notebook"
pio.renderers.default = "browser"

### Read data

In [2]:
from analysis.pipeline.full_pipeline import read_saved_data

_targets, actions, _metadata, idents, fixations, _visits = read_saved_data()

### Action-Time to Trial End

In [7]:
action_summary = (
    pd.concat([
        actions["to_trial_end"].describe().rename("all"),
        actions.groupby("subject")["to_trial_end"].describe().T,
    ], axis=1)
).T

action_summary

,count,mean,std,min,25%,50%,75%,max
all,1589.0,11057.115167,5808.963976,463.0,6382.00,10734.0,15974.00,27949.0
1,74.0,7020.689189,2810.501773,1093.0,5091.25,7470.5,9055.75,13270.0
2,121.0,12428.173554,4918.268616,1573.0,9135.00,12693.0,16286.00,21993.0
3,111.0,12330.171171,5410.932561,463.0,8110.50,12975.0,16979.00,20931.0
12,120.0,13161.583333,5338.944571,1111.0,9455.25,14438.5,17804.75,21913.0
13,106.0,7975.603774,3475.247274,755.0,5184.75,8065.5,10171.00,15734.0
14,110.0,13002.481818,5649.482481,902.0,8816.50,13818.0,17415.00,24478.0
15,107.0,7124.822430,5574.421230,565.0,1966.00,5158.0,11546.00,19429.0
16,93.0,13072.193548,6126.015025,1120.0,7447.00,14621.0,18084.00,23045.0
17,135.0,8600.511111,5372.653627,655.0,4188.50,7918.0,12254.00,26763.0


In [9]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        y0="to_trial_end", x=actions["to_trial_end"],
        name="All Subjects", legendgroup="All Subjects",
        text=actions.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in actions[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = actions[actions[cnfg.SUBJECT_STR] == subj_id]
    texts = subj_data.apply(
        lambda row: f"{subj_string}<br>"
                    f"Trial: {row['trial']}<br>"
                    # f"Target: {row['target']}<br>"
                    f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
        axis=1
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="to_trial_end", x=subj_data["to_trial_end"],
            text=texts,
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Identification-Action to Trial End", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()

### Identification-Fixation to Trial End

In [11]:
fixs_with_ident_time = fixations.copy()
dva_cols = [col for col in fixations.columns if col.endswith("distance_dva")]
fixs_with_ident_time["target"] = fixs_with_ident_time[dva_cols].idxmin(axis=1).str.replace("_distance_dva", "")
fixs_with_ident_time = (
    fixs_with_ident_time
    .drop(columns=[col for col in fixs_with_ident_time.columns if "_distance_" in col])
    .merge(
        idents.loc[
            idents["identification_category"] == "hit", ["subject", "trial", "target", "time"]
        ], on=["subject", "trial", "target"], how="left"
    )
)

fixs_with_ident_time.loc[:, "is_during"] = (fixs_with_ident_time["start_time"] <= fixs_with_ident_time["time"]) & (fixs_with_ident_time["time"] <= fixs_with_ident_time["end_time"])

In [13]:
ident_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_during"]]
ident_fixs_dist_summary = (
    pd.concat([
        ident_fixs["to_trial_end"].describe().rename("all"),
        ident_fixs.groupby("subject")["to_trial_end"].describe().T,
    ], axis=1)
).T

print("Identification Fixations:")
ident_fixs_dist_summary

Identification Fixations:


,count,mean,std,min,25%,50%,75%,max
all,2339.0,10606.000428,5675.217682,42.0,6207.5,10241.0,15447.00,26674.0
1,115.0,6388.113043,2823.822017,503.0,4419.0,7175.0,8601.50,11096.0
2,160.0,12230.331250,4905.685843,994.0,8690.0,12262.0,16264.00,21508.0
3,198.0,12011.929293,5167.474593,42.0,7856.0,12588.0,15902.00,20422.0
12,136.0,12656.698529,4867.801390,861.0,9707.0,13229.5,16977.25,19468.0
13,171.0,7471.730994,3220.888286,706.0,4768.5,7525.0,9608.50,15193.0
14,183.0,13117.836066,5082.810394,464.0,9665.0,13802.0,17144.50,24331.0
15,160.0,6376.681250,5621.097152,55.0,1111.0,4592.0,10753.50,18960.0
16,180.0,12625.611111,6002.550457,261.0,7177.5,14533.0,17366.00,22422.0
17,165.0,7840.890909,5228.716870,245.0,3695.0,6885.0,10963.00,26674.0


In [15]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, shared_yaxes=False,
)

# top: distribution across all subjects
fig.add_trace(
    row=1, col=1, trace=go.Violin(
        y0="to_trial_end", x=ident_fixs["to_trial_end"],
        name="All Subjects", legendgroup="All Subjects",
        text=ident_fixs.apply(
            lambda row: f"Subject: {row['subject']}<br>"
                        f"Trial: {row['trial']}<br>"
                        # f"Target: {row['target']}<br>"
                        f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
            axis=1
        ),
        marker=dict(color=cnfg.get_discrete_color("all")),
        width=1.75, orientation="h", side="positive", spanmode='hard',
        box=dict(visible=False),
        meanline=dict(visible=True),
        points="all", pointpos=-0.5,
        showlegend=True, hoverinfo="x+y+text",

    )
)

# bottom: distribution per subject
for subj_id in ident_fixs[cnfg.SUBJECT_STR].unique():
    subj_string = f"{cnfg.SUBJECT_STR.capitalize()} {subj_id:02d}"
    subj_data = ident_fixs[ident_fixs[cnfg.SUBJECT_STR] == subj_id]
    texts = subj_data.apply(
        lambda row: f"{subj_string}<br>"
                    f"Trial: {row['trial']}<br>"
                    # f"Target: {row['target']}<br>"
                    f"Time to Trial End: {row['to_trial_end']:.2f} DVA",
        axis=1
    )
    fig.add_trace(
        row=2, col=1, trace=go.Violin(
            y0="to_trial_end", x=subj_data["to_trial_end"],
            text=texts,
            name=subj_string, legendgroup=subj_string,
            marker=dict(color=cnfg.get_discrete_color(subj_id, loop=True), opacity=0.5),
            width=1.75, orientation="h", side="positive", spanmode='hard',
            box=dict(visible=False),
            meanline=dict(visible=True),
            points="all", pointpos=-0.5,
            showlegend=True, hoverinfo="x+y+text"
        )
    )

# update visuals
fig.update_annotations(font=cnfg.AXIS_LABEL_FONT)
fig.update_yaxes(showticklabels=False)  # Hide y-axis labels
fig.update_xaxes(
    title=None, showline=False,
    showgrid=True, gridcolor=cnfg.GRID_LINE_COLOR, gridwidth=cnfg.GRID_LINE_WIDTH,
    zeroline=False, zerolinecolor=cnfg.GRID_LINE_COLOR, zerolinewidth=cnfg.ZERO_LINE_WIDTH,
    tickfont=cnfg.AXIS_TICK_FONT,
)
fig.update_layout(
    width=1200, height=675,
    title=dict(text="Identification-Fixation to Trial End", font=cnfg.TITLE_FONT),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    # plot_bgcolor='rgba(0, 0, 0, 0)',
    showlegend=True,
)

fig.show()